In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

# ResNet 18

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.regularizers import l2
from keras.layers.merge import add



def conv_bn_relu(**conv_params):
    " builds: conv -> BN -> relu block "
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = layers.Conv2D(filters=filters, kernel_size=kernel_size,
                        strides=strides, padding=padding,
                        kernel_initializer=kernel_initializer,
                        kernel_regularizer=kernel_regularizer)(input)

        norm = layers.BatchNormalization(axis=-1)(conv)
        act = layers.Activation("relu")(norm)
        return act

    return f


def bn_relu_conv(**conv_params):
    """
    same as - conv_bn_relu - but with changed order
    builds: BN -> relu block ->  conv 
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    # kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        norm = layers.BatchNormalization(axis=-1)(input)
        act = layers.Activation("relu")(norm)
        conv = layers.Conv2D(filters=filters, kernel_size=kernel_size,
                        strides=strides, padding=padding,
                        kernel_initializer=kernel_initializer, #kernel_regularizer=kernel_regularizer
                        )(act)
        return conv

    return f



def basic_block(filters, init_strides=(1, 1)):

    def f(input):
        conv = bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                strides=init_strides)(input)

        residual = bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv)
        
        shortcut = input
        if init_strides == (2,2):
            shortcut = layers.Conv2D(filters=filters, kernel_size=(1, 1),
                          strides=init_strides,
                          padding="valid",
                          kernel_initializer="he_normal")(shortcut)
        output = add([shortcut, residual])
        return output

    return f



def basic_block_first(filters, init_strides=(1, 1)):

    def f(input):
        conv = layers.Conv2D(filters=filters, kernel_size=(3, 3),
                        strides=init_strides,
                        padding="same",
                        kernel_initializer="he_normal")(input)

        residual = bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv)
        
        shortcut = input
        if init_strides == (2,2):
            shortcut = layers.Conv2D(filters=filters, kernel_size=(1, 1),
                          strides=init_strides,
                          padding="valid",
                          kernel_initializer="he_normal")(shortcut)
        output = add([shortcut, residual])
        return output

    return f

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def resNet_18_encoder(input_shape=(224, 224, 3)):

    image_input = layers.Input(shape=(224, 224, 3))

    x = conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(image_input)  # output: 112x112
    f0 = x
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x)  # output: 56x56

    # BLOCK 1 - 64 filters
    x = basic_block_first(64, init_strides=(1, 1))(x) # output: 56x56
    x = basic_block(64, init_strides=(1, 1))(x)
    f1 = x

    # BLOCK 2 - 128 filters
    x = basic_block(128, init_strides=(2, 2))(x)  # output: 28x28
    x = basic_block(128, init_strides=(1, 1))(x)
    f2 = x

    # BLOCK 3 - 256 filters
    x = basic_block(256, init_strides=(2, 2))(x)  # output: 14x14
    x = basic_block(256, init_strides=(1, 1))(x)
    f3 = x

    # BLOCK 4 - 512 filters
    x = basic_block(512, init_strides=(2, 2))(x) # output: 7x7
    x = basic_block(512, init_strides=(1, 1))(x)
    f4 = x 


    return image_input, [f0, f1,f2,f3,f4]




# VGG16

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


from tensorflow.keras import layers

def vgg_encoder(input_shape=(224, 224, 3)):
    
    image_input = layers.Input(shape=(224, 224, 3))

    # Block 1
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(image_input)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    f1 = x
    
    # Block 2
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    f2 = x
    
    # Block 3
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    f3 = x

    # Block 4
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    f4 = x

    # Block 5
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    f5 = x
    
    return image_input, [f1,f2,f3,f4,f5]

# FPN

In [19]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

from tensorflow.keras.models import Model

def resnet_fpn(num_regions, input_height=224, input_width=224):

    model = fpn(resNet_18_encoder, num_regions, input_height=input_height, input_width=input_width)
    model.model_name = "resnet_fpn"
    
    return model


def fpn(encoder, num_regions, l1_skip_conn=True, input_height=224,
          input_width=224):

    image_input = layers.Input(shape=(224, 224, 3))
    mask = layers.Input(shape=(224, 224, 3))
    
    image_input, levels = encoder(image_input)
    [f0, f1,f2,f3,f4] = levels
    # paper says: f0 should be disregarded due to high memory footprint.

    # ***** FPN *****
    # 7x7 to 14x14
    P4 = f4
    f4_prime = layers.Conv2D(256, (1,1))(P4)
    x = layers.UpSampling2D((2, 2))(f4_prime)
    f3_prime = layers.Conv2D(256, (1,1))(f3)
    x = layers.Add()([x, f3_prime])
    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization(axis=-1)(x)
    P3 = layers.Activation("relu")(x)


    # 14x14 to 28x28
    x = layers.UpSampling2D((2, 2))(P3)
    f2_prime = layers.Conv2D(256, (1,1))(f2)
    x = layers.Add()([x, f2_prime])
    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization(axis=-1)(x)
    P2 = layers.Activation("relu")(x)


    # 28x28 to 56x56
    x = layers.UpSampling2D((2, 2))(P2)
    f1_prime = layers.Conv2D(256, (1,1))(f1)
    x = layers.Add()([x, f1_prime])
    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization(axis=-1)(x)
    P1 = layers.Activation("relu")(x)



    # ***** UPSAMPLE *****
    # upsample to 1/4 org. image
    # P4 - 1
    up_p4 = layers.Conv2D(128, (3, 3), padding='same')(P4)
    up_p4 = layers.BatchNormalization(axis=-1)(up_p4)  # tfa.layers.GroupNormalization()
    up_p4 = layers.Activation("relu")(up_p4)
    up_p4 = layers.UpSampling2D((2, 2))(up_p4)

    # P4 - 2
    up_p4 = layers.Conv2D(128, (3, 3), padding='same')(up_p4)
    up_p4 = layers.BatchNormalization(axis=-1)(up_p4)  # tfa.layers.GroupNormalization()
    up_p4 = layers.Activation("relu")(up_p4)
    up_p4 = layers.UpSampling2D((2, 2))(up_p4)

    # P4 - 1
    up_p4 = layers.Conv2D(128, (3, 3), padding='same')(up_p4)
    up_p4 = layers.BatchNormalization(axis=-1)(up_p4)  # tfa.layers.GroupNormalization()
    up_p4 = layers.Activation("relu")(up_p4)
    up_p4 = layers.UpSampling2D((2, 2))(up_p4)



    # P3 - 1
    up_p3 = layers.Conv2D(128, (3, 3), padding='same')(P3)
    up_p3 = layers.BatchNormalization(axis=-1)(up_p3)  # tfa.layers.GroupNormalization()
    up_p3 = layers.Activation("relu")(up_p3)
    up_p3 = layers.UpSampling2D((2, 2))(up_p3)

    # P3 - 2
    up_p3 = layers.Conv2D(128, (3, 3), padding='same')(up_p3)
    up_p3 = layers.BatchNormalization(axis=-1)(up_p3)  # tfa.layers.GroupNormalization()
    up_p3 = layers.Activation("relu")(up_p3)
    up_p3 = layers.UpSampling2D((2, 2))(up_p3)



    # P2 - 1
    up_p2 = layers.Conv2D(128, (3, 3), padding='same')(P2)
    up_p2 = layers.BatchNormalization(axis=-1)(up_p2)  # tfa.layers.GroupNormalization()
    up_p2 = layers.Activation("relu")(up_p2)
    up_p2 = layers.UpSampling2D((2, 2))(up_p2)

    # P2
    up_p1 = layers.Conv2D(128, (3, 3), padding='same')(P1)
    up_p1 = layers.BatchNormalization(axis=-1)(up_p1)  # tfa.layers.GroupNormalization()
    up_p1 = layers.Activation("relu")(up_p1)


    # SUM ELEMENT WISE
    y = layers.Add()([up_p4, up_p3, up_p2, up_p1])
    y = layers.UpSampling2D((4, 4))(y)


    # REGRESSION
    output_reg = layers.Conv2D(3, (3, 3), padding='same')(y)
    output_masked_reg = layers.Multiply()([output_reg, mask])

    
    # CLASSIFICATION
    x_clas = layers.Conv2D(num_regions, (3, 3), padding='same')(y)
    output_clas = keras.activations.softmax(x_clas, axis=-1)
    
    
    # CONCATENATE
    output = layers.Concatenate(axis=-1)([output_masked_reg, output_clas])
    
    
    model = Model(inputs=[image_input, mask], outputs=output) 
#     model.summary()
    
    return model


# UNET

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

from tensorflow.keras.models import Model

def resnet_unet(num_regions, input_height=224, input_width=224):

    model = unet(resNet_18_encoder, num_regions, input_height=input_height, input_width=input_width)
    model.model_name = "vgg_unet"
    
    return model

def vgg_unet(num_regions, input_height=224, input_width=224):

    model = unet(vgg_encoder, num_regions, input_height=input_height, input_width=input_width)
    model.model_name = "vgg_unet"
    
    return model


def unet(encoder, num_regions, l1_skip_conn=True, input_height=224,
          input_width=224):

    image_input = layers.Input(shape=(224, 224, 3))
    mask = layers.Input(shape=(224, 224, 3))
    
    image_input, levels = encoder(image_input)
    [f1, f2, f3, f4, f5] = levels

    o = f4
    o = (layers.ZeroPadding2D((1, 1), name='U_block1_zero_pad'))(o)
    o = (layers.Conv2D(512, (3, 3), padding='valid' , activation='relu', name='U_block1_conv'))(o)
    o = (layers.BatchNormalization(name='U_block1_batch_norm'))(o)

    o = (layers.UpSampling2D((2, 2), name='U_block2_up_sample'))(o)
    o = (layers.concatenate([o, f3], axis=-1, name='U_block2_concat'))
    o = (layers.ZeroPadding2D((1, 1)))(o)
    o = (layers.Conv2D(512, (3, 3), padding='valid', activation='relu'))(o)
    o = (layers.BatchNormalization())(o)

    o = (layers.UpSampling2D((2, 2)))(o)
    o = (layers.concatenate([o, f2], axis=-1))
    o = (layers.ZeroPadding2D((1, 1)))(o)
    o = (layers.Conv2D(256, (3, 3), padding='valid' , activation='relu'))(o)
    o = (layers.BatchNormalization())(o)

    o = (layers.UpSampling2D((2, 2)))(o)

    if l1_skip_conn:
        o = (layers.concatenate([o, f1], axis=-1))

    o = (layers.ZeroPadding2D((1, 1)))(o)
    o = (layers.Conv2D(128, (3, 3), padding='valid', activation='relu'))(o)
    o = (layers.BatchNormalization())(o)

    o = layers.Conv2D(128, (3, 3), padding='same')(o)
    
    o = (layers.UpSampling2D((2, 2)))(o)
    o = (layers.ZeroPadding2D((1, 1)))(o)
    o = (layers.Conv2D(64, (3, 3), padding='valid', activation='relu'))(o)
    o = (layers.BatchNormalization())(o)
    

    # REGRESSION
    output_reg = layers.Conv2D(3, (3, 3), padding='same')(o)
    output_masked_reg = layers.Multiply()([output_reg, mask])

    
    # CLASSIFICATION
    o_clas = layers.Conv2D(num_regions, (3, 3), padding='same')(o)
    output_clas = keras.activations.softmax(o_clas, axis=-1)
    
    
    # CONCATENATE
    output = layers.Concatenate(axis=-1)([output_masked_reg, output_clas])
    
    
    model = Model(inputs=[image_input, mask], outputs=output) 
#     model.summary()
    
    return model

# Memory requirements

In [11]:
from model_profiler import model_profiler
Batch_size = 32

### FPN + ResNet 18

In [18]:
model = resnet_fpn(141, input_height=224, input_width=224)

profile = model_profiler(model, Batch_size)

print(profile)

Running: VGG-16 backbone
| Model Profile                    | Value               | Unit    |
|----------------------------------|---------------------|---------|
| Selected GPUs                    | ['0']               | GPU IDs |
| No. of FLOPs                     | 0.28328173007999996 | BFLOPs  |
| GPU Memory Requirement           | 7.297399893403053   | GB      |
| Model Parameters                 | 15.292816           | Million |
| Memory Required by Model Weights | 58.33746337890625   | MB      |


### UNET + ResNet 18

In [16]:
model = resnet_unet(141, input_height=224, input_width=224)

profile = model_profiler(model, Batch_size)

print(profile)

| Model Profile                    | Value               | Unit    |
|----------------------------------|---------------------|---------|
| Selected GPUs                    | ['0']               | GPU IDs |
| No. of FLOPs                     | 0.44885446543999996 | BFLOPs  |
| GPU Memory Requirement           | 10.60492904484272   | GB      |
| Model Parameters                 | 8.923216            | Million |
| Memory Required by Model Weights | 34.03936767578125   | MB      |


### UNET + VGG16

In [17]:
model = vgg_unet(141, input_height=224, input_width=224)

profile = model_profiler(model, Batch_size)

print(profile)

| Model Profile                    | Value              | Unit    |
|----------------------------------|--------------------|---------|
| Selected GPUs                    | ['0']              | GPU IDs |
| No. of FLOPs                     | 0.7231124718399999 | BFLOPs  |
| GPU Memory Requirement           | 11.549675717949867 | GB      |
| Model Parameters                 | 15.688464          | Million |
| Memory Required by Model Weights | 59.84674072265625  | MB      |


In [21]:
(11.5- 7.29) / 11.5

0.3660869565217391

# ResNet 18 - github implementation
### implementation as presented in: https://github.com/raghakot/keras-resnet/blob/master/resnet.py
### I wanted to implement ResNet 18 myself and use this for comparison i.e. to check whether i have connected everything properly

In [27]:
from __future__ import division

import six
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten
)
from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D
)
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K


def _bn_relu(input):
    """Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)


def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f


def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f


def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks.
    """
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides,
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input

    return f


def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)

    return f


def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    Returns:
        A final conv layer of filters * 4
    """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
                              strides=init_strides,
                              padding="same",
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),
                                     strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        return _shortcut(input, residual)

    return f


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    ROW_AXIS = 1
    COL_AXIS = 2
    CHANNEL_AXIS = 3


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras `Model`.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        input_shape = (224,224,3)

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

#         Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1))(block)
        flatten1 = Flatten()(pool2)
        dense = Dense(units=num_outputs, kernel_initializer="he_normal",
                      activation="softmax")(flatten1)

        model = Model(inputs=input, outputs=block)
        return model

    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    @staticmethod
    def build_resnet_34(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_101(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])

    @staticmethod
    def build_resnet_152(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 8, 36, 3])


In [28]:
model = ResnetBuilder.build_resnet_18((224,224,3), 10)

In [12]:
# model.summary()